In [ ]:
if True:
  import nltk
  import re
  import os
  import pandas as pd
  import numpy as np
  import scipy.sparse as sp
  import lightgbm as lgb
  import scipy.sparse as sp

  from sklearn.preprocessing import LabelEncoder
  from sklearn.model_selection import StratifiedKFold
  from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
  from sklearn.linear_model import LogisticRegression, SGDClassifier
  from sklearn import metrics
  from sklearn.metrics import f1_score
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, classification_report
  from urllib.parse import unquote, urlparse, quote
  from sklearn.feature_extraction.text import TfidfVectorizer
  from tqdm import tqdm
  from sklearn import preprocessing
  from sklearn.preprocessing import MinMaxScaler

  import warnings
  warnings.filterwarnings('ignore')

  import seaborn as sns
  import matplotlib.pyplot as plt
  %matplotlib inline

  from pandas.api.types import is_datetime64_any_dtype as is_datetime
  from pandas.api.types import is_categorical_dtype
  import random

In [ ]:
def reduce_mem_usage(df, use_float16=False):
  """
  遍历数据帧的所有列并修改数据类型以减少内存使用。
  """

  start_mem = df.memory_usage().sum() / 1024**2
  print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

  for col in df.columns:
    if is_datetime(df[col]) or is_categorical_dtype(df[col]):
      continue
    col_type = df[col].dtype

    if col_type != object:
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == "int":
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)
      else:
        if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)
    else:
      df[col] = df[col].astype("category")

  end_mem = df.memory_usage().sum() / 1024**2
  print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
  print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

  return df

In [ ]:
# 数据读取
train = pd.read_csv('./data/dataTrain.csv')
no_label = pd.read_csv('./data/dataNoLabel.csv')
test = pd.read_csv('./data/dataA.csv')

if True:
  # 剔除最后1w个样本
  train = train[:50000]
else:
  # 剔除可能是假的样本
  fake_rows = train[train['id'] >= 170000].index.tolist()
  train.drop(fake_rows, inplace=True)
print(f'train dataset size: {len(train)}')

In [ ]:
if False:
  # 使用无标签数据集(手动打标签)
  no_label.drop(no_label[no_label['id'] >= 170000].index.tolist(), inplace=True)
  no_label['label'] = 0
  no_label[no_label['id'] < 42500] = 1

  rand_rows = random.sample([i for i in range(0, len(no_label))], 500)

  # train = pd.concat([train, no_label.iloc[rand_rows]]).reset_index(drop=True)
  train = pd.concat([train, no_label]).reset_index(drop=True)
  train = train.sample(frac=1.0).reset_index(drop=True) #shuffle
  train = train[:60000]
  print(f'train dataset size: {len(train)}')

In [ ]:
# 使用dataNoLabel.csv预测得到的样本
if False:
  train_extra = pd.read_csv('../data/09-29 11:41 0.909694498344 noLabelPred.csv', usecols=np.arange(48))
  i1 = train_extra['label'] < 0.95
  i2 = ~i1
  train_extra.loc[i1, 'label'] = 0
  train_extra.loc[i2, 'label'] = 1
  print(train_extra.shape)
  train_extra = train_extra[train_extra['label'] == 1]
  print(train_extra.shape)

  l0 = train['label'] == 0
  l1 = train['label'] == 1
  print(f'label=0: {sum(l0)}  label=1: {sum(l1)}')
  train = pd.concat([train, train_extra[:100]]).reset_index(drop=True)
  l0 = train['label'] == 0
  l1 = train['label'] == 1
  print(f'label=0: {sum(l0)}  label=1: {sum(l1)}')

  train_extra.head()

In [ ]:
# 类别特征列
categorical_f = ['f1','f2','f3']
# categorical_f = []

# 抛弃的特征列
drop_f = ['f11','f14','f16','f27','f28','f40','f41']
# drop_f = []
train.drop(drop_f, axis=1, inplace=True)
test.drop(drop_f, axis=1, inplace=True)

# 保留的原始特征列
original_f = [i for i in train.columns if i not in ['id','label']]

# 数据拼接
data = pd.concat([train, test]).reset_index(drop=True)
data['f3'] = data['f3'].map({'low': 0, 'mid': 1, 'high': 2})

In [ ]:
# 数据简单处理
cols = [i for i in original_f if i not in ['id','label']]
for i in cols:
  data[i] = data[i] + 1

In [ ]:
def getSegmentLength(col_name):
  if col_name in ['f4','f7','f8','f9','f10','f12','f13','f15','f17','f18','f19','f20','f21','f22','f23','f24','f25','f26','f29','f30',
                  'f31','f32','f33','f34','f35','f36','f37','f38','f39','f42']:
    return 30
  elif col_name in ['f5','f6']:
    return 50
  elif col_name in ['f43','f44','f45']:
    return 100
  elif col_name in ['f46']:
    return 200
  return 1

In [ ]:
# 统计转化率
if False:
  cols = [i for i in original_f if i not in ['id','label','f1','f2','f3']]
  one = train['label'] == 1
  idx1 = train['f5'] >= 60
  idx2 = train['f5'] < 90
  idx = idx1 & idx2
  idx = idx & one
  print(sum(idx))

In [ ]:
# 使用特征重要性文件
use_feat_rec = True

if use_feat_rec:
  feat_rec = pd.read_csv('f_importance.csv')
  feat_rec = feat_rec.sort_values(by='imp', ascending=False)
  feat_rec = feat_rec.reset_index()
  feat_rec = feat_rec[0:1000]

In [ ]:
# 加入frequency编码
if True:
  for col in tqdm(original_f):
    if col not in ['label', 'id']:
      data[col + '_cnt'] = data[col].map(data[col].value_counts())
  data.head()

In [ ]:
# 暴力组合位置特征
loc_f = ['f1', 'f2', 'f4', 'f5', 'f6']
for i in range(len(loc_f)):
  for j in range(i + 1, len(loc_f)):
    data[f'{loc_f[i]}+{loc_f[j]}'] = data[loc_f[i]] + data[loc_f[j]]
    data[f'{loc_f[i]}-{loc_f[j]}'] = data[loc_f[i]] - data[loc_f[j]]
    data[f'{loc_f[i]}*{loc_f[j]}'] = data[loc_f[i]] * data[loc_f[j]]
    data[f'{loc_f[i]}/{loc_f[j]}'] = data[loc_f[i]] / data[loc_f[j]]
    data[f'{loc_f[i]}={loc_f[j]}'] = data[loc_f[i]]*100000 + data[loc_f[j]]

# 暴力组合通话特征
com_f = ['f43', 'f44', 'f45', 'f46']
for i in range(len(com_f)):
  for j in range(i + 1, len(com_f)):
    data[f'{com_f[i]}+{com_f[j]}'] = data[com_f[i]] + data[com_f[j]]
    data[f'{com_f[i]}-{com_f[j]}'] = data[com_f[i]] - data[com_f[j]]
    data[f'{com_f[i]}*{com_f[j]}'] = data[com_f[i]] * data[com_f[j]]
    data[f'{com_f[i]}/{com_f[j]}'] = data[com_f[i]] / data[com_f[j]]
    data[f'{com_f[i]}={com_f[j]}'] = data[com_f[i]]*100000 + data[com_f[j]]
      
# 暴力组合互联网特征
net_f = [i for i in original_f if i not in ['label', 'id'] and i not in loc_f and i not in com_f]
for i in range(len(net_f)):
  for j in range(i + 1, len(net_f)):
    data[f'{net_f[i]}+{net_f[j]}'] = data[net_f[i]] + data[net_f[j]]
    data[f'{net_f[i]}-{net_f[j]}'] = data[net_f[i]] - data[net_f[j]]
    data[f'{net_f[i]}*{net_f[j]}'] = data[net_f[i]] * data[net_f[j]]
    data[f'{net_f[i]}/{net_f[j]}'] = data[net_f[i]] / data[net_f[j]]
    data[f'{net_f[i]}={net_f[j]}'] = data[net_f[i]]*100000 + data[net_f[j]]

for i in range(len(net_f)-2):
  for j in range(i + 1, len(net_f)-1):
    for k in range(j + 1, len(net_f)):
      data[f'{net_f[i]}&{net_f[j]}&{net_f[k]}'] = ((data[net_f[i]] * 7) + data[net_f[j]])*7 + data[net_f[k]]

# 组合特征重编码
# all_f = [i for i in data.columns if i not in ['label', 'id']]
# for i in all_f:
#   if '&' in i:
#     col = data[i]
#     uni = np.unique(col)
#     le = LabelEncoder()
#     le.fit(uni)
#     data[i] = le.transform(col)

In [ ]:
# 内存压缩
reduce_mem_usage(data)

# 训练测试分离
train = data[~data['label'].isna()].reset_index(drop=True)
test = data[data['label'].isna()].reset_index(drop=True)

Memory usage of dataframe is 4872.07 MB
Memory usage after optimization is: 1459.34 MB
Decreased by 70.0%


In [ ]:
# 特征筛选
if not use_feat_rec:
  features = [i for i in train.columns if i not in ['label', 'id']]
else:
  features = [i for i in train.columns if i in feat_rec['feat'].values or '_cnt' in i]

print(f'num featrues: {len(features)}')

num featrues: 1003


In [ ]:
# def scale_feature(X, column_names):
#   """标准化特征"""
#   print(column_names)
#   pre = preprocessing.scale(X[column_names])
#   X[column_names] = pre
#   return X
#
# train = scale_feature(train, features)
# test = scale_feature(test, features)

In [ ]:
# # 协方差 热力图
# _, dx = plt.subplots(figsize=(6,6))
# columns = train[features]
# sns.heatmap(train[features].corr(), annot=True, cmap='RdYlGn', ax=dx)

In [24]:
# 参数设定
y = train['label']
n_fold = 5;
KF = StratifiedKFold(n_splits=n_fold, random_state=2023, shuffle=True)
params = {
  'objective': 'binary',
  'boosting_type': 'gbdt',
  'metric': 'auc',
  'n_jobs': 30,
  'learning_rate': 0.04,
  'num_leaves': 31,
  # 'num_leaves': 2 ** 6, # 太多效果不好
  'max_depth': 6,
  'tree_learner': 'serial',
  'colsample_bytree': 0.8,
  'subsample_freq': 1,
  'subsample': 0.8,
  'num_boost_round': 3000,
  'max_bin': 255,
  'verbose': -1,
  'seed': 2022,
  'bagging_seed': 2022,
  'feature_fraction_seed': 2022,
  'early_stopping_rounds': 100
}

# 记录特征重要性
feat_imp = pd.DataFrame({'feat': features, 'imp': 0})

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros((len(test)))

# 模型训练
for fold, (trn_idx, val_idx) in enumerate(KF.split(train.values, y.values)):
  print(f'- Fold: {fold+1}')
  trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=y.iloc[trn_idx], categorical_feature=categorical_f)
  val_data = lgb.Dataset(train.iloc[val_idx][features], label=y.iloc[val_idx], categorical_feature=categorical_f)
  num_round = 3000
  clf = lgb.train(
    params,
    trn_data,
    num_round,
    valid_sets=[trn_data, val_data],
    verbose_eval=100,
    early_stopping_rounds=100,
  )
  print('\n')

  oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
  predictions_lgb[:] += clf.predict(test[features], num_iteration=clf.best_iteration) / n_fold
  feat_imp['imp'] += clf.feature_importance() / n_fold

auc = roc_auc_score(y, oof_lgb)
print('\nacu: %.12f' % auc)

- Fold: 1
[LightGBM] [Info] Number of positive: 9976, number of negative: 30024
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.304402 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 211322
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 1003
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.249400 -> initscore=-1.101815
[LightGBM] [Info] Start training from score -1.101815
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [ ]:
import time
import datetime
import pytz as pytz
tz = pytz.timezone('Asia/Shanghai')
t = datetime.datetime.fromtimestamp(int(time.time()), tz).strftime('%m-%d %H:%M')
print(t)

In [25]:
# 导出预测结果
test['label'] = predictions_lgb
test[['id', 'label']].to_csv('./%.12f.csv' %auc, index=False)

In [ ]:
# 导出特征重要性
if not use_feat_rec:
  feat_imp = feat_imp.sort_values(by='imp', ascending=False)
  feat_imp.to_csv('f_importance.csv', index=False)